<a href="https://colab.research.google.com/github/adrian-ja-projects/train-prediction-project/blob/fea_data_analisys/data_analysis_train_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/adrian-ja-projects/train-prediction-project.git

Cloning into 'train-prediction-project'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 41 (delta 25), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (41/41), done.


In [2]:
%run /content/train-prediction-project/pl_extraction_api_to_staging.ipynb

INFO: Creating list for extraction loop...
INFO: Extraction dates are between 2021-01-01 and 2021-04-10
INFO: Starting extraction...
INFO: Extraction completed a total of 100 file were successfully extracted into the staging area


In [3]:
%run /content/train-prediction-project/spark_dependencies.ipynb

INFO: Installing Spark dependencies...
INFO: Spark dependencies installed
INFO: env variables created


In [4]:
%run /content/train-prediction-project/pl_transf_staging_to_raw.ipynb

Data in raw ready to upload to a db


In [5]:
#create spark session
spark = SparkSession.builder\
        .master("local")\
        .appName("pl_data_analysis")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [9]:
raw_file_path = "/content/raw/digitraffic/27_schedule"
df_27_schedule = spark.read.format("parquet").load(raw_file_path)

In [8]:
#get average of the file
df_27_schedule.show()

+---------+--------------+-------------+-----------------+---------------+----------------+--------------------+-----------------------+------------+--------------+--------------------+----------------------+-----------------+-------------------+--------------+---------------+-----------+-------------------+---------------+--------------------+--------------------+----------------+--------------+--------+------+--------------------+-------------+---------+-----------------------+-------------+-------------+-----------+---------+------------+
|cancelled|commuterLineID|departureDate|operatorShortCode|operatorUICCode|runningCurrently|          actualTime|timeTableRows>cancelled|categoryCode|categoryCodeId|detailedCategoryCode|detailedCategoryCodeId|thirdCategoryCode|thirdCategoryCodeId|commercialStop|commercialTrack|countryCode|differenceInMinutes| estimateSource|    liveEstimateTime|       scheduledTime|stationShortCode|stationUICCode|accepted|source|           timestamp|trainStopping|  